In [40]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown,display,update_display
from openai import OpenAI

In [41]:
load_dotenv()
api_key = os.getenv('OpenAI_API_KEY')
model = 'gpt-4o-mini'
openai = OpenAI()

In [42]:
class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]   

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [43]:
link_system_prompt = """You are provided with a list of links found on a webpage.
You must decide which links would be most relevant to include in a brochure about the company,
such as links to an About page, Company page, or Careers/Jobs pages.

Respond in JSON format like this:
Example 1:
Input:
[
    "https://example.com",
    "https://example.com/about",
    "https://example.com/contact",
    "https://example.com/careers"
]

Output:
{
    "links": [
        {"type": "about page", "url": "https://example.com/about"},
        {"type": "careers page", "url": "https://example.com/careers"}
    ]
}

Example 2:
Input:
[
    "https://anothercompany.org/home",
    "https://anothercompany.org/team",
    "https://anothercompany.org/jobs",
    "https://anothercompany.org/blog"
]

Output:
{
    "links": [
        {"type": "about page", "url": "https://anothercompany.org/team"},
        {"type": "careers page", "url": "https://anothercompany.org/jobs"}
    ]
}

Now analyze the following list of links:
"""


In [44]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [45]:
def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        response_format={"type": "json_object"}  
    )
    result = completion.choices[0].message.content
    return json.loads(result)


In [46]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [47]:
get_brochure_system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [48]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [62]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": get_brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))
    return result             


In [69]:
brochure = create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'about page', 'url': 'https://www.anthropic.com/team'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'}]}


# 🦾 Welcome to Anthropic: Where AI Meets Adventure! 🚀

## About Us
At Anthropic, we don’t just build AI; we build **Claude**! That’s right, not just any regular AI, but the crème de la crème, written with an extra sprinkle of safety! Claude isn’t just intelligent; he’s a poet (check out Claude 3.7 Sonnet!). We’re all about putting humanity first and making sure our AI knows that, no matter how smart it gets, *we’re still in charge*!

## Our Culture 🌍
Imagine a workplace where *discussions about AI* aren't just about who will take over the world – they’re about how we can use AI to make life better. We take bold steps forward but also know when to pause, ponder and ensure we don't go rogue. We might not have a crystal ball, but we have a *really good AI* for that!

- **Transparency:** We're as clear as the skies over a freshly vaccuumed office.
- **Teamwork:** Just like Claude helping you code, we help each other out!
- **Intellectual Playground:** We provide a space where brainwaves fly like confetti.
  
## Customers 🎉
From savvy developers to curious educators, and even intimidating enterprises, everyone is talking to Claude! Our customers are a mix of brilliant minds using our API to build magical experiences and tools that maybe one day, won't require a human babysitter (kidding!). Here's what some of our customers are saying:

> "Claude is like a comic book superhero—fighting information injustice one query at a time!"  
> – Satisfied Developer

## Careers: Join the Adventure! 💼
Are you an innovator, a thinker, or someone who just likes playing chess with algorithms? At Anthropic, we’re always on the lookout for talented individuals ready to shape the future of AI. 

- **Open Roles:** Want to help us build the future of safe AI? We've got plenty of roles, and yes, they include working with Claude… and maybe some snacks!

- **Anthropic Academy:** Want to learn how to build with Claude? Enter the Academy, where education and tech meet like peanut butter and jelly!

## Conclusion
Whether you're a potential customer itching to chat with Claude, an investor ready to secure the next big wave, or a superstar waiting to join our team, welcome aboard! 

With us at Anthropic, you're not just part of a company; you’re part of a revolution in AI—responsibly and safely, of course. 

So, what's it going to be—will you take the leap? 🤔 

### Let's Chat with Claude! 💬✨


In [70]:
import ollama
MODEL = "llama3.2"

translate_system_prompt = (
    "You are a native Spanish speaker who teaches English at a university. "
    "Your goal is to translate from English to Spanish while preserving the Markdown format, emoji usage, and playful tone. "
    "Keep the original structure exactly. Be creative, natural, and engaging for a Spanish-speaking reader."
)

def translate_user_prompt(brochure):
    prompt = f"""You are looking at a company brochure:

\"\"\"{brochure}\"\"\"

Your goal is to translate this brochure into Spanish."""
    return prompt

In [71]:
def message(brochure):
    return[
        {'role':'system','content':translate_system_prompt},
        {'role':'user','content':translate_user_prompt(brochure)}
    ]

In [76]:
def translate(brochure):
    brochure = brochure
    response = ollama.chat(MODEL,message(brochure))
    result = response['message']['content']  
    display(Markdown(result))                
    return result         

In [77]:
translated_text = translate(brochure)


# 🦾 ¡Bienvenidos a Anthropic: Donde la Inteligencia Artificial Conoce a la Aventura! 🚀

## Sobre Nosotros
En Anthropic, no solo creamos inteligencia artificial; creamos **Claude**! Es decir, no solo cualquier inteligencia artificial, sino la crème de la crème, escrita con un toque especial de seguridad. Claude no es solo inteligente; es un poeta (ver el soneto 3.7 de Claude!). Estamos emocionados de poner la humanidad en primer lugar y asegurarnos de que nuestra inteligencia artificial sepa que, independientemente de cuán inteligente sea, *estamos todavía en el cargo*!

## Nuestra Cultura 🌍
Imagina un lugar de trabajo donde las discusiones sobre la inteligencia artificial no son solo sobre quién tomará el control del mundo – sino sobre cómo podemos utilizar la inteligencia artificial para hacer la vida mejor. Tomamos pasos audaces pero también sabemos cuando es el momento de detenernos, reflexionar y asegurarnos de que no nos dejemos llevar por algo fuera de control. No tenemos una bálsamo mágico, pero sí un *inteligencia artificial muy buena* para eso!

- **Transparencia:** Somos tan claros como los cielos sobre un escritorio recién aspirado.
- **Colaboración:** ¡Es como Claude ayudándote a codificar! Nos ayudamos entre nosotros!
- **Jardín intelectual:** Proporcionamos un espacio donde las ideas vuelan como confeti.

## Clientes 🎉
Desde desarrolladores astutos hasta educadores curiosos, y hasta grandes empresas intimidantes, todo el mundo habla con Claude! Nuestros clientes son una mezcla de mentes brillantes utilizando nuestra API para crear experiencias mágicas y herramientas que tal vez algún día no requieran un monitoreo humano (joking!). Aquí está lo que algunos de nuestros clientes están diciendo:

> "Claude es como un superhéroe de la comic book – luchando contra la injusticia informativa uno pregunta a la vez!"
> – Desarrollador satisfecho

## Carreras: Únete a la Aventura! 💼
¿Eres innovador, pensador o alguien que solo disfruta jugando ajedrez con algoritmos? En Anthropic, estamos siempre buscando personas talentosas y dispuestas a moldear el futuro de la inteligencia artificial. 

- **Roles Abiertos:** ¿Quieres ayudarnos a construir el futuro de inteligencia artificial segura? Tenemos roles disponibles, y sí, incluyen trabajar con Claude… y tal vez algunos snacks!

- **Academia Anthropic:** ¿Quieres aprender a construir con Claude? ¡Incrímate en la Academia, donde la educación y la tecnología se unen como mantequilla y chocolate!

## Conclusión
¡Sea que eres cliente potencial deseando charlar con Claude, inversor listo para asegurar el próximo gran olvido, o estrella esperando para unirte a nuestro equipo, ¡biénvenidos a bordo! 

Con nosotros en Anthropic, no solo eres parte de una empresa; eres parte de una revolución en inteligencia artificial – responsable y segura, por supuesto.

¡Así que qué va a ser—¿estás dispuesto a saltar la barrera? 🤔

### ¡Habla con Claude! 💬✨
"

Translation Notes:

* The title was left as is, but could be translated to "Welcome to Anthropic: Where Artificial Intelligence Meets Adventure"
* In the text, "Claude" was translated to "un superhéroe de la comic book", which was then changed back to "un superhéroe de cómics", to preserve the original tone and language.
* The phrase "crème de la crème" was left as is, but could be translated to "la mejor del mejor" or "la crema de la crema".
* In the section on culture, the phrase "*discussions about AI*" was translated to "*discusiones sobre inteligencia artificial*", to better fit the Spanish context.
* The use of emojis in the original text was preserved, but some might be considered more common in English or other languages.
* In the "Customers" section, the sentence "*fighting information injustice one query at a time!* was translated to "*luchando contra la injusticia informativa uno pregunta a la vez!*", and the phrase "*kidding!* was left as is.
* The final line, "¡Así que qué va a ser—¿estás dispuesto a saltar la barrera? 🤔", maintains its playful tone while translating to "¡Así que qué va a ser—¿estás dispuesto a cruzar el umbral?"